In [51]:
!pip3 install geopy

import geopandas as gpd
import pandas as pd
import matplotlib as plt
import folium
import pyproj
from shapely.geometry import Point, Polygon, LineString
import geopy.distance

  Using cached geopy-2.3.0-py3-none-any.whl (119 kB)
  Using cached geographiclib-2.0-py3-none-any.whl (40 kB)

[notice] A new release of pip available: 22.3 -> 23.1.2
[notice] To update, run: pip3 install --upgrade pip


In [23]:
stat = pd.read_csv('../station_coordinate.csv')

In [24]:
stat = stat.drop(columns = ['code', 'line'])
stat

,name,lat,lng
0,녹양,37.759380,127.042292
1,남영,37.541021,126.971300
2,용산,37.529849,126.964561
3,노량진,37.514219,126.942454
4,대방,37.513342,126.926382
...,...,...,...
725,지식정보단지,37.378384,126.645168
726,인천대입구,37.386007,126.639484
727,센트럴파크,37.393054,126.634729
728,국제업무지구,37.399907,126.630347


In [46]:
station = gpd.GeoDataFrame(
    stat, geometry=gpd.points_from_xy(stat.lng, stat.lat), crs="EPSG:3310"
)
station

,name,lat,lng,geometry
0,녹양,37.759380,127.042292,POINT (127.042 37.759)
1,남영,37.541021,126.971300,POINT (126.971 37.541)
2,용산,37.529849,126.964561,POINT (126.965 37.530)
3,노량진,37.514219,126.942454,POINT (126.942 37.514)
4,대방,37.513342,126.926382,POINT (126.926 37.513)
...,...,...,...,...
725,지식정보단지,37.378384,126.645168,POINT (126.645 37.378)
726,인천대입구,37.386007,126.639484,POINT (126.639 37.386)
727,센트럴파크,37.393054,126.634729,POINT (126.635 37.393)
728,국제업무지구,37.399907,126.630347,POINT (126.630 37.400)


In [67]:
ddarung = pd.read_csv('../따릉이정보_수정.csv', encoding = 'euc-kr')
ddarung = ddarung.drop(columns = ['Unnamed: 0'])
ddarung['shortest_subway'] = ""
ddarung['shortest_subway_meter'] = ""
ddarung

,보관소(대여소)명,자치구,상세주소,위도,경도,shortest_road,법정동,shortest_subway,shortest_subway_meter
0,경복궁역 7번출구 앞,종로구,서울특별시 종로구 사직로 지하130 경복궁역 7번출구 앞,37.575794,126.971451,사직로12길,내자동,,
1,경복궁역 4번출구 뒤,종로구,서울특별시 종로구 사직로 지하130 경복궁역 4번출구 뒤,37.575947,126.974060,사직로,적선동,,
2,광화문역 1번출구 앞,종로구,서울특별시 종로구 세종대로 지하189 세종로공원,37.571770,126.974663,세종대로23길,당주동,,
3,종로구청 옆,종로구,서울특별시 종로구 세종로 84-1,37.572559,126.978332,종로1길,세종로,,
4,서울역사박물관 앞,종로구,서울특별시 종로구 새문안로 55 서울역사박물관 앞,37.570000,126.971100,경희궁1길,신문로2가,,
...,...,...,...,...,...,...,...,...,...
2714,명일역 3번출구 옆,강동구,강동구 명일동 303-7,37.551426,127.144211,양재대로,명일동,,
2715,상일파출소 옆,강동구,강동구 상일동 520-1,37.556553,127.165878,고덕로79길,상일동,,
2716,강변그대가리버뷰 입구 맞은편,강동구,강동구 천호동 250-7,37.548164,127.122574,선사로,천호동,,
2717,방아삼거리,강동구,강동구 둔촌동 118-2,37.523281,127.146133,동남로43길,둔촌동,,


In [60]:
def getShorestDistanceStation(point): #subway
    distances = station.geometry.distance(point)
    return station.iloc[distances.idxmin()]['name'], distances.min(), distances.idxmin(), station.iloc[distances.idxmin()]['lat'], station.iloc[distances.idxmin()]['lng'] 


In [68]:
import warnings
warnings.filterwarnings(action='ignore')

for index, row in ddarung.iterrows():
    output = getShorestDistanceStation(Point(row['경도'], row['위도']))
    dist = geopy.distance.geodesic((row['위도'], row['경도']),(output[3],output[4]))

    ddarung['shortest_subway'][index] = output[0]
    ddarung['shortest_subway_meter'][index] = round(dist.meters, 1)

    print(str(index) + " " + row['보관소(대여소)명'] + " -> " + output[0])
    print(round(dist.meters, 1))    
ddarung

0  경복궁역 7번출구 앞 -> 경복궁
183.7
1  경복궁역 4번출구 뒤 -> 경복궁
51.1
2  광화문역 1번출구 앞 -> 광화문
195.5
3  종로구청 옆 -> 광화문
224.9
4  서울역사박물관 앞 -> 광화문
505.0
5  광화문 S타워 앞 -> 광화문
268.3
6  광화문역 6번출구 옆 B -> 광화문
126.5
7  국립현대미술관 -> 안국
541.0
8  종각역 1번출구 앞 -> 종각
103.6
9  안국역 5번출구 앞 -> 안국
68.6
10  낙원상가 옆 -> 안국
389.7
11  탑골공원 앞 -> 종로3가
346.6
12  청계2가 사거리 옆 -> 을지로3가
430.8
13  종로3가역 2번출구 뒤 -> 종로3가
111.9
14  종로3가역 15번출구 앞 -> 종로3가
163.7
15  창경궁 입구 -> 혜화
603.1
16  세운스퀘어 앞 -> 을지로4가
453.7
17  종로4가 사거리 -> 종로5가
323.5
18  혜화동 로터리 -> 혜화
379.6
19  혜화역 3번출구 뒤 -> 혜화
85.2
20  대학로 마로니에공원 -> 혜화
289.7
21  예일빌딩(율곡로) 앞 -> 종로5가
577.8
22  성균관대 E하우스 앞 -> 동대문
394.5
23  서울보증보험본사 앞 -> 종로5가
282.8
24  독립문역 사거리 -> 독립문
389.3
25  사직동주민센터 -> 경복궁
441.0
26  청운초교 앞 삼거리 -> 경복궁
1065.6
27  중앙고입구 삼거리 -> 안국
770.4
28  재동초교 앞 삼거리 -> 안국
326.0
29  포르투갈 대사관 앞 -> 안국
430.0
30  서울사대부속초교 앞 -> 종로5가
621.8
31  KT혜화지사 앞 -> 혜화
576.5
32  성대입구 사거리 -> 혜화
292.8
33  원남동사거리 -> 종로5가
678.6
34  동묘앞역 1번출구 뒤 -> 동묘앞
82.4
35  청계8가 사거리 -> 신설동
400.8
36  신설동역 11번출구 뒤 -> 신설동
200.3
37  창신역

297  뚝섬역 1번 출구 옆 -> 뚝섬
258.6
298  뚝섬역 5번 출구 정류소 옆 -> 뚝섬
236.4
299  성수사거리 버스정류장 앞 -> 건대입구
646.5
300  청계천 박물관 앞 -> 용두
368.2
301  양지사거리 -> 상왕십리
297.1
302  상왕십리역 4번 출구 앞 -> 상왕십리
101.5
303  왕십리역 11번 출구 앞 -> 왕십리
248.7
304  금호역 1번출구 앞 -> 금호
83.3
305  옥수동성당 옆 -> 금호
510.8
306  래미안금호하이리버 아파트 102동 옆 -> 신금호
492.1
307  한양대병원사거리 -> 한양대
424.6
308  용답토속공원 앞 -> 답십리
505.2
309  장한평역 8번 출구 앞 -> 장한평
85.5
310  청계벽산아파트 앞 -> 상왕십리
496.5
311  금남시장 교차로 -> 금호
461.1
312  금호사거리 -> 신금호
811.3
313  응봉삼거리 -> 응봉
456.1
314  행당역 2번출구 앞 -> 행당
23.4
315  한양대후문역 부근 -> 왕십리
349.2
316  답십리역 8번출구 앞 -> 답십리
169.6
317  서울시설공단 앞 -> 용두
341.1
318  달맞이공원 -> 옥수
263.5
319  도선동 주민센터 앞 -> 상왕십리
487.5
320  성동광진 교육지원청 앞 -> 왕십리
357.4
321  왕십리역 4번 출구 건너편 -> 왕십리
95.8
322  비전교회 앞 -> 어린이대공원
617.6
323  서울숲역 2번출구 앞 -> 서울숲
88.9
324  군자지하보도 앞 -> 군자
590.9
325  성동세무서 건너편 -> 성수
704.9
326  금호역 3번출구 -> 금호
29.0
327  옥수역 3번출구 -> 옥수
77.8
328  성수역 2번출구 앞 -> 성수
99.2
329  청계8가사거리 부근 -> 신설동
484.7
330  응봉현대아파트 정류소 -> 응봉
381.7
331  성동세무서 부근 -> 성수
706.8
332  마장역 4번출구

601 서울중랑우체국 -> 상봉
202.9
602 면목삼익아파트 앞 -> 용마산
680.5
603 홈플러스 상봉점 앞 -> 망우
139.4
604 면목 홈플러스 2 -> 사가정
564.3
605  용마지구대 옆 -> 사가정
740.3
606  새우개마을 -> 신내
367.6
607  상봉2동 주민센터 앞 -> 중랑
443.4
608  동서울농협 앞 -> 상봉
49.7
609  라코지스테이 앞 -> 중랑
217.2
610  신내2동 주민센터 앞 -> 망우
784.6
611  동성1,2차 아파트 앞 -> 신내
679.8
612  한양수자인아파트 앞 -> 사가정
505.4
613  묵동 이마트 앞 -> 먹골
288.2
614  망우지구대 -> 망우
404.9
615  중랑데시앙플렉스 앞 -> 신내
522.5
616  면목동 새싹어린이공원 앞 -> 사가정
512.6
617  망우역 2번 출구 -> 망우
80.0
618  방정환교육지원센터 옆 -> 상봉
341.7
619  신내동 벽산아파트 후문 앞 -> 양원
763.5
620  신내동 한살림 중랑지구 앞 -> 망우
713.1
621  신내동 한살림 중랑지구 북축 -> 망우
730.3
622  상봉신협 앞 -> 면목
586.9
623  홈플러스 면목점 -> 사가정
547.6
624  신내SK V1센터 앞 -> 양원
818.4
625  용마복지센터 앞 -> 면목
716.3
626  망우동 본죽 앞 -> 양원
743.3
627  양원역 1번 출구 -> 양원
72.6
628  양원지구 힐데스하임 앞 -> 양원
329.7
629 동원사거리 행복오피스텔 앞 -> 면목
604.6
630  서울시 북부병원 앞 -> 양원
287.6
631  신내 우디안 111동 앞 -> 신내
676.7
632 중화수경공원 -> 중화
458.4
633  한성대입구역6번출구 뒤 -> 한성대입구
34.9
634  돈암초교 입구 -> 성신여대입구
360.1
635  만해공원 -> 한성대입구
1394.6
636  성북구청 -> 성신여대입구
365.7
637 

895  노원경찰서교차로 -> 중계
677.7
896  인덕대학교 -> 월계
555.8
897  상계동수락리버시티 -> 도봉산
638.6
898  서울북부고용센터앞 -> 노원
338.8
899  월계문화체육센터 -> 월계
768.8
900  육군사관학교 앞 -> 봉화산
657.7
901  성서대학교 밀알관 -> 중계
428.0
902  삼육대 도서관 -> 갈매
1202.0
903  삼육대 제3과학관 -> 갈매
1101.9
904  현대6차 아파트 -> 하계
1639.0
905  중계종합사회복지관 교차로 -> 상계
417.9
906  노원문화예술회관 -> 상계
1324.3
907  태릉입구역 5번출구 -> 태릉입구
58.6
908  불암고등학교 앞 횡단보도 -> 상계
1534.6
909  상계주공10단지 종합상가 앞 -> 마들
423.0
910  마들역 7번출구 -> 마들
86.7
911  마들역 3번출구 -> 마들
39.3
912  도봉운전면허시험장 버스정류장 옆 -> 노원
341.1
913  노원정보도서관 -> 노원
711.3
914  온곡초교 교차로 -> 상계
761.7
915  원자력 병원 -> 화랑대
930.1
916 수락리버시티4단지 -> 도봉산
664.1
917 상계주공2단지 219동 -> 노원
657.0
918 공릉동풍림아파트 101동 -> 하계
737.4
919 공릉풍림아파트 108동 -> 공릉
617.5
920 월계초안아파트 1단지 -> 녹천
1371.6
921 중계3차청구아파트 -> 상계
1151.9
922  월계중학교 -> 월계
574.4
923  광운대학교 중앙도서관 -> 광운대
495.7
924  석계역 3번 출구 -> 석계
158.9
925 도깨비시장 -> 공릉
426.9
926 상계역중앙시장 -> 상계
266.5
927 대진고등학교 -> 하계
506.9
928 상계8동 주민센터 -> 마들
588.3
929 서울과학기술대학교(미래관) -> 공릉
831.6
930 서울과학기술대학교(어학교육원) -> 하계
869.9
931 신창중학교 후문

1181  마포소방서 앞 -> 광흥창
276.9
1182  신성기사식당 앞 -> 대흥
328.1
1183  서강대 정문 건너편 -> 서강대
139.7
1184  서강대 남문 옆 -> 대흥
334.9
1185  서강대 후문 옆 -> 대흥
332.5
1186  현대벤처빌 앞 -> 신촌
179.2
1187  신촌역(2호선) 6번출구 옆 -> 신촌
60.4
1188  신촌역(2호선) 7번출구 앞 -> 신촌
71.8
1189  아현역 4번출구 앞 -> 아현
44.9
1190  공덕역 2번출구 -> 공덕
136.4
1191  공덕역 8번출구 -> 공덕
54.1
1192  공덕역 5번출구 -> 공덕
26.1
1193  마포역 2번출구 뒤 -> 마포
41.3
1194  마포역 4번출구 뒤 -> 마포
33.5
1195  용강동 주민센터 앞 -> 마포
401.0
1196  서강대역 2번출구 앞 -> 서강대
169.9
1197  성산2교 사거리 -> 가좌
491.7
1198  마포구청역 -> 마포구청
346.1
1199  가좌역1 번출구 앞 -> 가좌
44.6
1200  서울서부지방법원 앞 -> 애오개
516.9
1201  애오개역 4번출구 앞 -> 애오개
82.4
1202  망원초록길 입구 -> 망원
838.7
1203  망원2빗물펌프장 앞 -> 망원
810.6
1204  하늘채코오롱아파트 건너편 -> 가좌
505.4
1205  7번가피자 서교망원점 -> 망원
376.3
1206  마포 신수공원 앞 -> 광흥창
581.8
1207  월드컵공원 -> 마포구청
456.4
1208  서울 월드컵 경기장 -> 월드컵경기장
390.4
1209  상암한화오벨리스크 1차 앞 -> 수색
1338.0
1210  상암월드컵파크 10단지 앞 -> 수색
1371.3
1211  상암월드컵파크 9단지 앞 -> 수색
1412.4
1212  부엉이공원 앞 -> 수색
1096.3
1213  DMC빌 앞 -> 수색
890.7
1214  상암월드컵파크 7단지 앞 -> 수색
1302.0
1215  마포구 육아종합

...

1767  진미파라곤 앞 -> 국회의사당
463.1
1768  국민일보 앞 -> 국회의사당
593.6
1769  국회의사당역 3번출구 옆 -> 국회의사당
73.3
1770  국회의사당역 5번출구 옆 -> 국회의사당
75.5
1771  산업은행 앞 -> 국회의사당
309.8
1772  KBS 앞 -> 국회의사당
381.9
1773  여의나루역 1번출구 앞 -> 여의나루
88.7
1774  유진투자증권빌딩 앞 -> 여의도
462.3
1775  여의도역 4번출구 옆 -> 여의도
77.8
1776  여의도역 1번출구 옆 -> 여의도
70.7
1777  KT앞 -> 신길
554.1
1778  금융감독원 앞 -> 여의도
334.6
1779  여의도고교 앞 -> 여의나루
307.2
1780  NH농협은행 앞 -> 샛강
560.4
1781  롯데캐슬엠파이어 옆 -> 여의도
178.2
1782  미성아파트 A동 앞 -> 여의도
299.8
1783  여의도초교 앞 -> 여의나루
654.1
1784  시범아파트버스정류장 옆 -> 노량진
720.7
1785  진주아파트상가 앞 -> 샛강
476.6
1786  롯데캐슬 앞 -> 샛강
468.1
1787  앙카라공원 앞 -> 샛강
74.2
1788  샛강역 1번출구 앞 -> 샛강
54.5
1789  양평2나들목 보행통로 입구 -> 신목동
468.5
1790  선유도역 3번출구 앞 -> 선유도
99.7
1791  양평1 보행육교 앞 -> 선유도
266.2
1792  영등포구청역 1번출구 -> 영등포구청
44.0
1793  남부고용노동지청 남측 -> 영등포구청
368.5
1794  양평우림 이비즈센타 앞 -> 영등포구청
722.9
1795  양평1동 YP센터 앞 -> 영등포구청
545.1
1796  영등포구민체육센터 앞 -> 신풍
363.6
1797  신길동 우리은행 옆 -> 신풍
498.6
1798  문래동자이아파트 앞 -> 문래
197.7
1799  제2구민체육센타 앞 -> 영등포시장
463.2
1800  유스호스텔 앞 -> 영등포시장

2060  신봉천주유소 B -> 낙성대
514.3
2061 신사로입구 -> 신림
680.2
2062  KT관악지점 -> 봉천
475.5
2063  낙성대역 5번출구 -> 낙성대
105.3
2064  난곡사거리 -> 신대방
628.8
2065  광일빌딩 -> 사당
429.1
2066  관악 GSfresh -> 서울대입구
378.9
2067  은천교회 앞 -> 신대방
738.9
2068  서울대벤처타운역 -> 신림
1382.3
2069  은천삼거리 -> 서울대입구
487.3
2070  본마을 정류소 앞 -> 청계산입구
768.6
2071 청계산입구역 1번출구 -> 청계산입구
222.9
2072  청계산입구역 2번출구 -> 청계산입구
75.9
2073  내곡3단지 어린이공원 앞 -> 청계산입구
796.8
2074  언남초등학교 앞 -> 청계산입구
976.6
2075  내곡파출소 뒤 정자 -> 청계산입구
1144.3
2076  반포1동 주민센터 앞 -> 사평
214.4
2077  서초구립반포도서관 앞 -> 사평
313.3
2078 서초현대4차아파트 201동 맞은편 -> 사평
493.6
2079  고속터미널역 5번출구 앞 -> 고속터미널
371.8
2080  신반포역 2번출구 앞 -> 신반포
158.7
2081  반포종합운동장 입구 -> 신반포
193.1
2082 아크로리버뷰 부지 앞 -> 잠원
466.3
2083  고속터미널역 8-1번, 8-2번 출구 사이 -> 고속터미널
177.6
2084  방일초교 마을버스 정류장 -> 내방
245.6
2085 사당역 12번출구 뒤 -> 사당
173.6
2086  방배래미안 정문 앞 -> 사당
450.5
2087 백석예술대학교 제3캠퍼스 앞 -> 방배
876.0
2088  신동아럭스빌아파트 앞 -> 방배
770.0
2089 방배열린문화센터 앞 -> 내방
181.6
2090 뒷벌공원 옆 -> 총신대입구
574.2
2091 로고스교회 맞은 편 -> 총신대입구
275.8
2092 이수역 4번 출구 -> 총신대입구
74.3
2093  삼

2356  세곡데이케어센터 -> 복정
2345.0
2357  신청담나들목 -> 청담
966.2
2358  일원1동주민센터앞 사거리 -> 일원
881.7
2359  대청역 -> 대청
46.1
2360  LH강남힐스테이트아파트 -> 일원
1491.5
2361  더시그넘하우스 앞 -> 수서
1639.1
2362  상록수아파트 -> 일원
365.5
2363  선정릉역 4번 출구 -> 선정릉
151.3
2364  구룡역 1번 출구 -> 구룡
61.6
2365  구룡역 4번 출구 -> 구룡
146.2
2366  구룡역 5번 출구 -> 구룡
90.5
2367  개포동역 4번출구 -> 개포동
20.1
2368 KT&G타워 옆(상상마당 대치아트홀) -> 삼성
259.2
2369  구역삼세무서 교차로 -> 역삼
650.4
2370  신사스퀘어 -> 신사
330.4
2371  강남경찰서 -> 삼성
335.8
2372  섬유센터 앞 -> 삼성
294.4
2373  극동스포츠 앞 -> 압구정
272.4
2374  SB타워 앞 -> 압구정로데오
686.9
2375  노보텔 앰배서더 앞 -> 신논현
385.9
2376  도곡동 골프 로데오거리 -> 양재
781.1
2377  레베쌍트 빌딩 앞 -> 삼성중앙
278.2
2378  도심공항타워 앞 -> 봉은사
494.8
2379  영동중앙교회 버스정류장 -> 역삼
369.4
2380  청담역 1번출구 -> 청담
384.8
2381 일원에코파크 주차장 -> 대청
644.1
2382 매봉역 1번출구 -> 매봉
117.5
2383 학여울역사거리(은마아파트) -> 학여울
107.2
2384  가락시장역 3번 출구 -> 가락시장
78.0
2385  밀리아나2빌딩 앞 -> 가락시장
250.0
2386  거여역 3번출구 -> 거여
57.7
2387  종합운동장역 4번출구 -> 종합운동장
210.3
2388  9호선종합운동장역 9번출구 -> 종합운동장
407.7
2389  마천중앙시장 -> 마천
529.5
2390  풍납현대아파트쉼터 -> 강동구청
81.

2660 강동경희대학교병원 입구 -> 고덕
324.2
2661 (시립)고덕평생학습관 -> 고덕
342.0
2662 명일근린공원공동체텃밭 -> 고덕
514.7
2663 고덕역 5번출구 -> 고덕
40.8
2664  SSTS 몰 앞 -> 길동
382.6
2665 대명초교입구 사거리 -> 굽은다리
577.6
2666 천동초교 삼거리 -> 길동
467.3
2667  천동초교입구사거리 버스정류장(십자성마을 방면) -> 길동
249.6
2668 강동역 1번출구 앞 -> 강동
45.1
2669 둔촌사거리 -> 둔촌동
685.5
2670 강일동입구교차로 -> 상일동
691.0
2671 엘웨딩 -> 강동
376.8
2672 트레빌빌딩 -> 강동
292.4
2673 중앙보훈병원역3번출구 -> 길동
1212.5
2674 광문고교사거리(아이파크) -> 고덕
633.2
2675  윤선생빌딩(JYP사옥) -> 올림픽공원
884.6
2676 래미안 솔베뉴(고덕로) -> 명일
391.8
2677 래미안 솔베뉴(구천면로) -> 명일
154.5
2678 풍납사거리(강동) -> 강동구청
261.2
2679 고덕 래미안힐스테이트(201동) -> 고덕
905.2
2680 상일동역 2번출구 앞 -> 상일동
25.5
2681  서울천호동우체국 -> 천호
324.3
2682  프라이어팰리스아파트 -> 명일
548.1
2683  명일초등학교 옆 -> 명일
386.0
2684  명일동신동아아파트 앞 -> 고덕
214.5
2685  암사정수센터교차로(강동롯데캐슬퍼스트아파트) -> 명일
786.6
2686  맥도날드 서울둔촌DT점 앞 -> 둔촌동
211.3
2687  둔촌신성미소지움아파트 -> 둔촌동
541.1
2688  고덕센트럴 아이파크상가 앞 -> 상일동
685.9
2689  둔촌푸르지오아파트 -> 길동
605.4
2690  한영외국어고등학교 -> 고덕
707.5
2691  강동리버스트4단지아파트 404동 앞 -> 상일동
1761.5
2692  강동리버스트4단지아파트 405동 앞 -> 상일동
1794.1

,보관소(대여소)명,자치구,상세주소,위도,경도,shortest_road,법정동,shortest_subway,shortest_subway_meter
0,경복궁역 7번출구 앞,종로구,서울특별시 종로구 사직로 지하130 경복궁역 7번출구 앞,37.575794,126.971451,사직로12길,내자동,경복궁,183.7
1,경복궁역 4번출구 뒤,종로구,서울특별시 종로구 사직로 지하130 경복궁역 4번출구 뒤,37.575947,126.974060,사직로,적선동,경복궁,51.1
2,광화문역 1번출구 앞,종로구,서울특별시 종로구 세종대로 지하189 세종로공원,37.571770,126.974663,세종대로23길,당주동,광화문,195.5
3,종로구청 옆,종로구,서울특별시 종로구 세종로 84-1,37.572559,126.978332,종로1길,세종로,광화문,224.9
4,서울역사박물관 앞,종로구,서울특별시 종로구 새문안로 55 서울역사박물관 앞,37.570000,126.971100,경희궁1길,신문로2가,광화문,505.0
...,...,...,...,...,...,...,...,...,...
2714,명일역 3번출구 옆,강동구,강동구 명일동 303-7,37.551426,127.144211,양재대로,명일동,명일,19.7
2715,상일파출소 옆,강동구,강동구 상일동 520-1,37.556553,127.165878,고덕로79길,상일동,상일동,50.8
2716,강변그대가리버뷰 입구 맞은편,강동구,강동구 천호동 250-7,37.548164,127.122574,선사로,천호동,암사,495.8
2717,방아삼거리,강동구,강동구 둔촌동 118-2,37.523281,127.146133,동남로43길,둔촌동,둔촌동,1006.9


In [63]:
ddarung.to_csv('../따릉이정보_수정2.csv', encoding='euc-kr')